In [1]:
from pynq import Overlay


In [2]:
ol = Overlay("usb_microblaze.bit")


In [3]:
customIOP = {
    'ip_name': ol.CustomIOP.description["memories"]["axi_bram_ctrl_0"]["fullpath"], #'CustomIOP/axi_bram_ctrl_0',
    'rst_name': "xlslice_0", #'mb_iop_pmoda_reset',
    'intr_pin_name': "CustomIOP/dff_en_reset_vector_0/q", #'iop_pmoda/dff_en_reset_vector_0/q'
    'intr_ack_name': "xlslice_1"#'mb_iop_pmoda_intr_ack'
    }

In [4]:
from pynq.lib import PynqMicroblaze

In [5]:
MAILBOX_OFFSET = 0xF000
MAILBOX_SIZE = 0x1000
MAILBOX_PY2IOP_CMD_OFFSET = 0xffc
MAILBOX_PY2IOP_ADDR_OFFSET = 0xff8
MAILBOX_PY2IOP_DATA_OFFSET = 0xf00

TEST = 0x69
SEND = 0x9
INIT = 0x23

ARDUINO_ADDRESS = 0x02 

In [6]:
import time
class MB(PynqMicroblaze):
    def __init__(self, mb_info, mb_program):
        super().__init__(mb_info, mb_program)

    def write_mailbox(self, data_offset, data):
        offset = MAILBOX_OFFSET + data_offset
        self.write(offset, data)

    def read_mailbox(self, data_offset, num_words=1):
        offset = MAILBOX_OFFSET + data_offset
        return self.read(offset, num_words)

    def write_blocking_command(self, command):
        self.write(MAILBOX_OFFSET + MAILBOX_PY2IOP_CMD_OFFSET, command)
        while self.read(MAILBOX_OFFSET + MAILBOX_PY2IOP_CMD_OFFSET) != 0:
            pass
    def write_blocking_command_addr(self, addr, command):
        self.write(addr, command)
        while self.read(addr) != 0:
            pass        

    def write_non_blocking_command(self, command):
        self.write(MAILBOX_OFFSET + MAILBOX_PY2IOP_CMD_OFFSET, command)

In [50]:
mb_info = customIOP

_mb = MB(mb_info, "usb_microblaze.bin")

In [51]:
_mb.write_mailbox(0, 0)
_mb.write_blocking_command(TEST)
_mb.read_mailbox(0x0)

69

In [52]:
_mb.write_mailbox(0, 4)
_mb.write_blocking_command(INIT)
_mb.read_mailbox(0x0)

4

In [71]:
for i in range(0,10):
    time.sleep(1)
    _mb.write_mailbox(0, 87) # 'W'
    _mb.write_mailbox(4, 1)
    _mb.write_blocking_command(SEND)
    _mb.read_mailbox(0x0)